In [4]:
import os
import json
import logging
import inspect
import importlib.util
from google import genai
from google.genai import types
from dotenv import load_dotenv, find_dotenv
from typing import Union, get_type_hints
import atexit
import gradio as gr

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


c:\Users\achakravarti\AppData\Local\anaconda3\envs\general\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-28 15:41:39,880 - INFO - HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"


In [18]:
# Function registry (automatically populated)
function_registry = {}

def load_functions_from_directory(directory: str):
    """
    Scans a directory and its subdirectories for Python files, imports them, and registers functions.

    Args:
        directory (str): The directory containing function modules.
    """
    global function_registry

    if not os.path.isdir(directory):
        logging.error(f"Directory {directory} does not exist!")
        return

    for root, _, files in os.walk(directory):  # Recursively walk through subdirectories
        for filename in files:
            if filename.endswith(".py") and not filename.startswith("__"):
                module_name = filename[:-3]  # Remove '.py' extension
                module_path = os.path.join(root, filename)

                # Dynamically load the module
                spec = importlib.util.spec_from_file_location(module_name, module_path)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)

                # Register only functions that start with "ai_"
                for name, func in inspect.getmembers(module, inspect.isfunction):
                    if name.startswith("ai_"):  # Naming convention check
                        function_registry[name] = func

                logging.info(f"Loaded functions from {filename}: {list(function_registry.keys())}")

load_functions_from_directory("functions")

2025-02-28 16:13:32,175 - INFO - Loaded functions from file_ops.py: ['ai_create_folders', 'ai_get_file_list', 'ai_get_unique_file_types', 'ai_move_files_to_folder']


In [21]:
def extract_function_metadata():
    """
    Extracts function names and signatures dynamically.

    Returns:
        str: JSON-formatted string containing function metadata.
    """
    json_list = [
        {"name": name, "signature": str(inspect.signature(func))}
        for name, func in function_registry.items()
    ]
    
    return json.dumps(json_list, indent=2)  # Pretty-print JSON for readability
        
func_def_n_info = extract_function_metadata()
print(func_def_n_info)

[{"name": "ai_create_folders", "signature": "(unique_file_types: Set[str], base_path: str = '.') -> Dict[str, str]"}, {"name": "ai_get_file_list", "signature": "(path: str) -> List[str]"}, {"name": "ai_get_unique_file_types", "signature": "(file_list: List[str]) -> Set[str]"}, {"name": "ai_move_files_to_folder", "signature": "(source_folder: str, folder_paths: Dict[str, str]) -> None"}]


In [20]:
function_registry

{'ai_create_folders': <function file_ops.ai_create_folders(unique_file_types: Set[str], base_path: str = '.') -> Dict[str, str]>,
 'ai_get_file_list': <function file_ops.ai_get_file_list(path: str) -> List[str]>,
 'ai_get_unique_file_types': <function file_ops.ai_get_unique_file_types(file_list: List[str]) -> Set[str]>,
 'ai_move_files_to_folder': <function file_ops.ai_move_files_to_folder(source_folder: str, folder_paths: Dict[str, str]) -> None>}

In [ ]:
function_registry